#### Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import seaborn as sns

#### Importando dataset - TUSS 30726034: Artroplastia de Joelho

In [2]:
dataset = pd.read_csv('data/data1.csv', encoding = "ISO-8859-1", delimiter='\',\'', engine='python')

#### Procedimento para agrupar os serviços por id_conta, em formato de string para utilizá-lo no count e no tf-idf.

#### Variáveis:
servicos: lista com todos os serviços existentes no dataset.<br>
serv: lista com os serviços em formato de string, agrupados por id_conta.<br>
contas: ordem das contas utilizadas em serv.

In [3]:
data = list(dataset.servico.groupby(dataset['id_conta']))
servicos = list(dataset.servico.astype(str).value_counts().index)
serv = []
contas = []
for i in range(len(data)):
    serv.insert(i, ' '.join(map(str,data[i][1])))
    contas.append(str(data[i][0]))

A idéia por trás desse método é a de que cada linha do dataset seja uma id_conta e cada coluna um dos serviços existentes no dataset, com seu valor representativo para a conta em questão, ou o número de vezes que aparece na string representando a id_conta(método count) ou baseado na frequência do serviço nessa conta e na frequência com que ele aparece em todo o dataset(método tf-idf).

In [4]:
# exemplo variavel serv(string com todos os serviços de uma conta)
serv[50]

'90196031 90090845 90090845 90090845 40304590 40304590 40304590 90196503 90196503 90196031 90196503 40302237 40302237 40302237 40301931 40301931 40301931 70135355 70135355 70135355 0000084866 0000084866 0000084866 70222800 70222800 70222800 70223769 70223769 70223769 40301222 40301222 40301222 40304639 40304639 40304639 40302423 40302423 40302423 70714100 70714100 70714100 01213730 01213730 01213730 01213731 01213731 01213731 01213734 01213734 01213734 01213732 01213732 01213732 01213733 01213733 01213733 60001046 60001046 60001046 80010660 80010660 80010660 60023155 60023155 60023155 60031808 60031808 60031808 60004355 60004355 60004355 60029269 60029269 60029269 90006330 90006330 90006330 90003551 90003551 90003551 90045769 90045769 90045769 90015738 90015738 90015738 90045980 90045980 90045980 40301397 40301397 40301397 40302512 40302512 40302512 40302504 40302504 40302504 40301630 40301630 40301630 40302580 40302580 40302580 40302040 40302040 40302040 40811026 40811026 40811026 201

#### Utilizando tf-idf(Term Frequency - Inverse Document Frequency) para retornar valores normalizados referente aos serviços utilizados por id_conta. A matriz resultante e seus atributos serão utilizados na clusterização.

#### Utilizando count para contar o número de vezes que cada serviço aparece na conta. A matriz resultante será utilizada após a clusterização.

In [5]:
tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, vocabulary=servicos)
X = tfidf.fit_transform(serv).toarray()
count = CountVectorizer(min_df=0, vocabulary=servicos)
X_count = count.fit_transform(serv).toarray()
X.shape

/usr/lib64/python3.5/site-packages/sklearn/feature_extraction/text.py:993: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(float(n_samples) / df) + 1.0


(2453, 11489)

X.shape indica que o dataset possui 2453 contas diferentes e 11489 serviços distintos.

In [6]:
#exemplo de X
X[50]

array([ 0.02756953,  0.        ,  0.04862406, ...,  0.        ,
        0.        ,  0.        ])

#### A seguir é usado o método de clusterização por hierarquia, com a métrica de distância escolhida sendo a maior 'cosine distance' entre os elementos do cluster.

O valor da distância por esse método será entre 0(são similares) e 1(totalmente distintos).

In [7]:
Z = linkage(X, method='complete', metric='cosine')

### Visualizando Z:
Primeira coluna: posição da conta 1.<br>
Segunda coluna: posição da conta 2.<br>
Terceira coluna: distância entre as contas.<br>
Quarta coluna: quantidade de pontos no cluster criado através da junção das duas contas.

In [8]:
Z[:20]

array([[  0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          2.00000000e+00],
       [  2.00000000e+00,   2.45300000e+03,   0.00000000e+00,
          3.00000000e+00],
       [  3.00000000e+00,   2.45400000e+03,   0.00000000e+00,
          4.00000000e+00],
       [  6.00000000e+00,   2.45500000e+03,   0.00000000e+00,
          5.00000000e+00],
       [  4.00000000e+00,   5.00000000e+00,   0.00000000e+00,
          2.00000000e+00],
       [  2.07400000e+03,   2.45700000e+03,   0.00000000e+00,
          3.00000000e+00],
       [  7.00000000e+00,   2.45600000e+03,   0.00000000e+00,
          6.00000000e+00],
       [  1.00000000e+01,   2.45900000e+03,   0.00000000e+00,
          7.00000000e+00],
       [  1.40000000e+01,   2.46000000e+03,   0.00000000e+00,
          8.00000000e+00],
       [  1.20000000e+01,   4.42000000e+02,   0.00000000e+00,
          2.00000000e+00],
       [  1.54000000e+02,   4.18000000e+02,   0.00000000e+00,
          2.00000000e+00],
       [  6.12000000e

#### Criando os clusters utilizando como métrica a distância máxima entre os pontos de 0.3(70% de similaridade).

In [9]:
clusters = fcluster(Z, 0.3, 'distance')
clusters

array([414, 414, 414, ..., 609, 482,   6], dtype=int32)

#### Número de clusters criados:

In [10]:
clusters.max()

777

#### Criando dataframe do resultado dos clusters.

In [11]:
result = pd.DataFrame(X_count, columns=servicos)
contas = pd.DataFrame(contas, columns=['conta'])
result.set_index(contas.conta, inplace=True)
result['cluster'] = clusters
result.head(5)

,30726034,70014370,90196031,70705330,40304361,20203047,40301630,40302580,40302318,40302423,...,09142302,0000074439,0000040730,00165316,90050045,0000008639,00607201,00201299,70138990,cluster
conta,,,,,,,,,,,,,,,,,,,,,
1135485394,16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,414
1135517427,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,414
1135517428,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,414
1135644291,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,414
1135853444,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,392


#### Procurando cluster com mais contas atreladas.

In [12]:
result.cluster.value_counts()[:10]

414    1361
116      36
381      25
412      17
436      11
75       10
424       9
409       8
391       8
396       8
Name: cluster, dtype: int64

Como pode-se ver a maior parte das contas estão em um cluster (414), provavelmente são as contas que possuem apenas o TUSS Master. E grande parte dos clusters tem menos de 8 contas, mostrando serem contas com serviços únicos que não aparecem nas demais.

#### Imprimindo o número de serviços existentes nos 10 maiores clusters e a contagem de vezes que cada serviço aparece.

In [13]:
for cluster in result.cluster.value_counts()[:10].index:
    print('Cluster: %d' % (cluster))   
    for servico in list(filter(lambda x: x != 'cluster', result.columns)):
        if(result[servico].loc[result.cluster==cluster].max() > 0):
            print('Servico: %s,  Count: %.2f' % (servico, result[servico].loc[result.cluster==cluster].sum()))
    print('--------------------------------')

Cluster: 414
Servico: 30726034,  Count: 4060.00
--------------------------------
Cluster: 116
Servico: 30726034,  Count: 725.00
Servico: 40403408,  Count: 149.00
Servico: 40403173,  Count: 122.00
Servico: 40403343,  Count: 142.00
Servico: 40403688,  Count: 149.00
Servico: 65410254,  Count: 135.00
--------------------------------
Cluster: 381
Servico: 30726034,  Count: 397.00
Servico: 65410254,  Count: 89.00
--------------------------------
Cluster: 412
Servico: 30726034,  Count: 36.00
Servico: 10104020,  Count: 1.00
Servico: 10104011,  Count: 1.00
Servico: 30732026,  Count: 30.00
--------------------------------
Cluster: 436
Servico: 30726034,  Count: 27.00
Servico: 30726220,  Count: 3.00
Servico: 30726239,  Count: 24.00
--------------------------------
Cluster: 75
Servico: 30726034,  Count: 10.00
Servico: 30726220,  Count: 12.00
Servico: 30732026,  Count: 10.00
Servico: 30713072,  Count: 2.00
--------------------------------
Cluster: 424
Servico: 30726034,  Count: 9.00
Servico: 307271